In [23]:
import pandas as pd, numpy as np

# 파일 불러오기
file_search = "C:/Users/cogus/Documents/seminar2025/datalab_HLB.xlsx"
file_stock  = "C:/Users/cogus/Documents/seminar2025/신성델타테크.xlsx"

# 파일 읽기
search_raw = pd.read_excel(file_search, sheet_name="개요")
stock_raw  = pd.read_excel(file_stock,  sheet_name="Sheet1")

C:\Users\cogus\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [29]:
# 검색량 날짜/볼륨 필터링
search_trim = pd.read_excel("datalab_신성.xlsx", sheet_name="개요", header=6)
search_trim = search_trim.rename(columns={"날짜":"date", "신성델타테크":"search_raw"})
search_trim["date"] = pd.to_datetime(search_trim["date"])
search_trim["search_raw"] = pd.to_numeric(search_trim["search_raw"], errors="coerce")
print(search_trim.head())

        date  search_raw
0 2023-07-16     0.31694
1 2023-07-17     1.47350
2 2023-07-18     1.07731
3 2023-07-19     0.90055
4 2023-07-20     1.37597


C:\Users\cogus\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [31]:
# 주가/거래량 필터링
stock_trim = stock_raw.rename(columns={"일자":"date","거래량":"volume","종가":"close"}).copy()
stock_trim["date"] = pd.to_datetime(stock_trim["date"], format="%Y/%m/%d", errors="coerce")
stock_trim["volume"] = (
    stock_trim["volume"].astype(str).str.replace(",", "", regex=False)
    .astype(float).astype("Int64")
)
stock_trim["close"] = pd.to_numeric(stock_trim["close"], errors="coerce")
stock_trim = stock_trim.dropna(subset=["date","volume"]).sort_values("date")

In [33]:
print(stock_trim.head())

          date  close   volume
103 2023-07-17  12700  2118062
102 2023-07-18  12230   931258
101 2023-07-19  12170   696135
100 2023-07-20  13190  2753471
99  2023-07-21  13110   738183


In [35]:
# 거래일 기준으로 데이터 병합
merged = (stock_trim[["date","close","volume"]]
          .merge(search_trim, on="date", how="inner")
          .sort_values("date").reset_index(drop=True))

# 로그차분 (Granger용)
eps = 1e-9
merged["log_volume"] = np.log(merged["volume"] + eps)
merged["log_search"] = np.log(merged["search_raw"] + eps)
merged["dlog_volume"] = merged["log_volume"].diff()
merged["dlog_search"] = merged["log_search"].diff()

granger_ready = merged.dropna(subset=["dlog_volume","dlog_search"]).reset_index(drop=True)

In [39]:
print(merged.head())
print(granger_ready.head())

        date  close   volume  search_raw  log_volume  log_search  dlog_volume  \
0 2023-07-17  12700  2118062     1.47350   14.566012    0.387641         <NA>   
1 2023-07-18  12230   931258     1.07731   13.744292    0.074467     -0.82172   
2 2023-07-19  12170   696135     0.90055   13.453299   -0.104750    -0.290993   
3 2023-07-20  13190  2753471     1.37597   14.828373    0.319159     1.375074   
4 2023-07-21  13110   738183     1.04836   13.511947    0.047227    -1.316426   

   dlog_search  
0          NaN  
1    -0.313173  
2    -0.179217  
3     0.423909  
4    -0.271932  
        date  close   volume  search_raw  log_volume  log_search  dlog_volume  \
0 2023-07-18  12230   931258     1.07731   13.744292    0.074467     -0.82172   
1 2023-07-19  12170   696135     0.90055   13.453299   -0.104750    -0.290993   
2 2023-07-20  13190  2753471     1.37597   14.828373    0.319159     1.375074   
3 2023-07-21  13110   738183     1.04836   13.511947    0.047227    -1.316426   
4 2023

In [43]:
# 데이터 저장~ 했는데 어디갔지 ㅅㅂ
merged.to_csv("신성델타테크_검색량_거래량_머지_raw.csv", index=False)
granger_ready.to_csv("신성델타테크_검색량_거래량_Granger용_dlog.csv", index=False)

In [41]:
# (번외)디렉토리 설정만 잘 해도 뭔가 해결이 되는 경우가 많음
import os
os.getcwd()

'C:\\Users\\cogus\\Documents\\seminar2025'

In [17]:
os.chdir("C:/Users/cogus/Documents/seminar2025")
print(os.getcwd())

C:\Users\cogus\Documents\seminar2025
